In [1]:
import numpy as np
import pandas as pd
import cv2
import random
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mostCommon import most_common
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math
from sklearn.ensemble import RandomForestRegressor

In [2]:
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
data_array = my_data.to_numpy()
images_path = '../data/Five_Frames_Per_patient/'

my_convex=pd.read_csv('../data/convex_image_data.csv')
convex_path='../data/pocus_images/convex/'

convex_data=[]
images_data = []
models = []

In [3]:
X_train=[]
X_test=[]
y_train=[]
y_test=[]

In [4]:
df=my_convex
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(convex_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img_1d = img.reshape(224 * 224)
    convex_data.append(img_1d)
convex_data=np.asarray(convex_data)
convex_y=np.asarray(my_convex['Label'])

In [5]:
df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)
X=np.asarray(images_data)
print(type(X))
y = np.asarray(my_data['Label'])

<class 'numpy.ndarray'>


In [6]:
X=np.concatenate((X,convex_data),0)
y=np.concatenate((y,convex_y),0)
print('Applying pca to the data')
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set
X = scaler.transform(X)
pca = PCA(n_components=270)
pca.fit(X)
X = pca.transform(X)

Applying pca to the data


In [7]:
print(len(y))

639


In [1]:
l=0
ll=[0,1,2,3,4]
ll=[t+l for t in ll]
print(ll)
X_test=np.take(X,ll,axis=0)
y_test=np.take(y,ll)
X_train=np.delete(X,ll,axis=0)
y_train=np.delete(y,ll)
print(len(X_test))
print(len(y_test))
print(len(X_train))
print(len(y_train))

[0, 1, 2, 3, 4]


NameError: name 'np' is not defined

In [9]:
print(y_test[0])

covid


In [10]:
r = []
conf_matricies = []
f1_reports = []
f1_micro = []
total_actual=[]
total_predicted=[]
l=5
ll=[-5,-4,-3,-2,-1]
for o in range(122):
    print('---------------------------- Patient ' + str(o) + ' ------------------------------ \n')
    ll=[t+l for t in ll]
    print('Indexes: '+str(ll))
    X_test=np.take(X,ll,axis=0)
    y_test=np.take(y,ll)
    X_train=np.delete(X,ll,axis=0)
    y_train=np.delete(y,ll)
    r=[155]
    print('Random seeds are: '+str(r))

    number_of_models = 1
    X = np.asarray(images_data)
    y = np.asarray(my_data['Label'])
    Xandy = []
    models = []
    # creating models
    for i in range(number_of_models):
        models.append(make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=r[0])))

    # Creating array test splits for models
    # X_train, X_test, y_train, y_test
    for i in range(number_of_models):
        #append x train x test y train ytest to x
        x=[]
        x.append(X_train)
        x.append(X_test)
        x.append(y_train)
        x.append(y_test)
        Xandy.append(x)

    print('Equalizing Data...')
    for i in range(number_of_models):
        x = Xandy[i][0]
        y1 = Xandy[i][2]
        cov = np.count_nonzero(y1 == 'covid')
        pneu = np.count_nonzero(y1 == 'pneumonia')
        reg = np.count_nonzero(y1 == 'regular')
        # deleting random pneumonia elements from each model set
        while cov != pneu:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 'pneumonia':
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                pneu -= 1
        # deleting random regular elements from each model set
        while cov != reg:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 'regular':
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                reg -= 1
        Xandy[i][0] = x
        Xandy[i][2] = y1
    
    print('Training Models...')
    for i in range(number_of_models):
        models[i].fit(Xandy[i][0], Xandy[i][2])

    print('getting predictions of each model...')
    predicted = []
    for i in range(number_of_models):
        predicted.append(models[i].predict(Xandy[i][1]))

    voted_prediction = []
    for i in range(len(predicted[0])):
        voting = []
        for j in range(len(predicted)):
            voting.append(predicted[j][i])
        voted_prediction.append(most_common(voting))
    total_actual.append(Xandy[0][3])
    total_predicted.append(voted_prediction)
    print(voted_prediction)
    print(Xandy[0][3])
print('All Done :)')

---------------------------- Patient 0 ------------------------------ 

Indexes: [0, 1, 2, 3, 4]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia']
['covid' 'covid' 'covid' 'covid' 'covid']
---------------------------- Patient 1 ------------------------------ 

Indexes: [5, 6, 7, 8, 9]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['covid', 'covid', 'covid', 'covid', 'covid']
['covid' 'covid' 'covid' 'covid' 'covid']
---------------------------- Patient 2 ------------------------------ 

Indexes: [10, 11, 12, 13, 14]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['covid', 'covid', 'covid', 'covid', 'covid']
['covid' 'covid' 'covid' 'covid' 'covid']
---------------------------- Patient 3 ------------------------------ 

Indexes: [15, 16, 17, 18, 19]
Random seeds are: [155]

Training Models...
getting predictions of each model...
['covid', 'covid', 'covid', 'covid', 'covid']
['covid' 'covid' 'covid' 'covid' 'covid']
---------------------------- Patient 29 ------------------------------ 

Indexes: [145, 146, 147, 148, 149]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['covid', 'covid', 'covid', 'covid', 'covid']
['covid' 'covid' 'covid' 'covid' 'covid']
---------------------------- Patient 30 ------------------------------ 

Indexes: [150, 151, 152, 153, 154]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia']
['pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia']
---------------------------- Patient 31 ------------------------------ 

Indexes: [155, 156, 157, 158, 159]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['pneumonia', 'pneu

Equalizing Data...
Training Models...
getting predictions of each model...
['pneumonia', 'pneumonia', 'regular', 'pneumonia', 'pneumonia']
['pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia']
---------------------------- Patient 54 ------------------------------ 

Indexes: [270, 271, 272, 273, 274]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'regular', 'pneumonia', 'pneumonia', 'pneumonia']
['pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia']
---------------------------- Patient 55 ------------------------------ 

Indexes: [275, 276, 277, 278, 279]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia']
['pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia']
---------------------------- Patient 56 ------------------------------ 

Indexes: [280, 281, 282, 283, 284]
Random seeds are: [155]
E

Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'regular', 'regular', 'regular', 'regular']
['regular' 'regular' 'regular' 'regular' 'regular']
---------------------------- Patient 80 ------------------------------ 

Indexes: [400, 401, 402, 403, 404]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'regular', 'regular', 'regular', 'regular']
['regular' 'regular' 'regular' 'regular' 'regular']
---------------------------- Patient 81 ------------------------------ 

Indexes: [405, 406, 407, 408, 409]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'regular', 'regular', 'regular', 'regular']
['regular' 'regular' 'regular' 'regular' 'regular']
---------------------------- Patient 82 ------------------------------ 

Indexes: [410, 411, 412, 413, 414]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictio

Training Models...
getting predictions of each model...
['regular', 'regular', 'regular', 'regular', 'regular']
['regular' 'regular' 'regular' 'regular' 'regular']
---------------------------- Patient 106 ------------------------------ 

Indexes: [530, 531, 532, 533, 534]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'regular', 'regular', 'regular', 'regular']
['regular' 'regular' 'regular' 'regular' 'regular']
---------------------------- Patient 107 ------------------------------ 

Indexes: [535, 536, 537, 538, 539]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model...
['regular', 'regular', 'regular', 'regular', 'regular']
['regular' 'regular' 'regular' 'regular' 'regular']
---------------------------- Patient 108 ------------------------------ 

Indexes: [540, 541, 542, 543, 544]
Random seeds are: [155]
Equalizing Data...
Training Models...
getting predictions of each model

In [11]:
patients_p=[]
actual=[]
for i in range(len(total_predicted)):
    patients_p.append(most_common(total_predicted[i]))
    actual.append(most_common(total_actual[i]))
i=0
j=0
correct=0
while i < len(total_actual):
    if patients_p[i]==actual[i]:
        correct+=1
    i+=1
correct

99

In [14]:
from collections import Counter

a = dict(Counter(patients_p))
b=dict(Counter(actual))
print(a)
b

{'pneumonia': 43, 'covid': 27, 'regular': 52}


{'covid': 30, 'pneumonia': 36, 'regular': 56}

In [15]:
#Patient confusion matrix and frames confusion matrix
print('------------------Patient Level confusion matrix and report-------------------')
print(confusion_matrix(patients_p,actual))
print(metrics.classification_report(patients_p, actual, digits=3))

------------------Patient Level confusion matrix and report-------------------
[[21  1  5]
 [ 6 32  5]
 [ 3  3 46]]
              precision    recall  f1-score   support

       covid      0.700     0.778     0.737        27
   pneumonia      0.889     0.744     0.810        43
     regular      0.821     0.885     0.852        52

    accuracy                          0.811       122
   macro avg      0.803     0.802     0.800       122
weighted avg      0.818     0.811     0.812       122



In [105]:
t=[]
p=[]
for i in total_predicted:
    for j in i:
        t.append(j)
for i in total_actual:
    for j in i:
        p.append(j)
print('-------------Frame level Confusion matrix and report-----------------')
print(confusion_matrix(t,p))
print(metrics.classification_report(t, p, digits=3))

-------------Frame level Confusion matrix and report-----------------
[[115   8  27]
 [ 22 154  29]
 [ 13  18 224]]
              precision    recall  f1-score   support

       covid      0.767     0.767     0.767       150
   pneumonia      0.856     0.751     0.800       205
     regular      0.800     0.878     0.837       255

    accuracy                          0.808       610
   macro avg      0.807     0.799     0.801       610
weighted avg      0.810     0.808     0.807       610



In [17]:
print(patients_p)
print(actual)

['pneumonia', 'covid', 'covid', 'regular', 'covid', 'regular', 'covid', 'covid', 'covid', 'covid', 'covid', 'covid', 'covid', 'pneumonia', 'covid', 'covid', 'covid', 'covid', 'covid', 'covid', 'pneumonia', 'covid', 'covid', 'regular', 'pneumonia', 'pneumonia', 'covid', 'pneumonia', 'covid', 'covid', 'pneumonia', 'pneumonia', 'covid', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'regular', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'regular', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'regular', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'pneumonia', 'covid', 'pneumonia', 'covid', 'covid', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'pneumonia', 'covid', 'regular', 're